# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [48]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [49]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [50]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2431797693970,
 'week52high': 162.16,
 'week52low': 108.32,
 'week52highSplitAdjustOnly': 157.64,
 'week52lowSplitAdjustOnly': 107.65,
 'week52change': 0.17205108745156264,
 'sharesOutstanding': 17213602163,
 'float': 0,
 'avg10Volume': 73340966,
 'avg30Volume': 84413240,
 'day200MovingAvg': 139.81,
 'day50MovingAvg': 147.25,
 'employees': 153826,
 'ttmEPS': 5.1,
 'ttmDividendRate': 0.8619773943915486,
 'dividendYield': 0.006197332428107678,
 'nextDividendDate': '',
 'exDividendDate': '2021-08-03',
 'nextEarningsDate': '2021-10-21',
 'peRatio': 27.563041355228197,
 'beta': 1.406289432410269,
 'maxChangePercent': 55.14594902968963,
 'year5ChangePercent': 4.214159207967221,
 'year2ChangePercent': 1.47695194030284,
 'year1ChangePercent': 0.17398889916286375,
 'ytdChangePercent': 0.0690096462885252,
 'month6ChangePercent': 0.0525734558224202,
 'month3ChangePercent': -0.031362956524408105,
 'month1ChangePercent': -0.05943415041148531,
 'day30Change

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [51]:
data['year1ChangePercent']

0.17398889916286375

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [52]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [53]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True
        )
final_dataframe.head()

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,156.580,0.434027,N/A
1,AAL,20.071,0.621434,N/A
2,AAP,218.320,0.366884,N/A
3,AAPL,143.120,0.175359,N/A
4,ABBV,108.800,0.302439,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [54]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,23,ALB,236.050,1.409502,N/A
1,98,CMA,85.550,1.142082,N/A
2,66,BKR,27.400,1.139239,N/A
3,89,CF,63.570,1.109087,N/A
4,29,AMAT,131.020,1.048395,N/A
5,82,CBRE,104.190,1.045079,N/A
6,18,AIG,60.010,0.937281,N/A
7,24,ALGN,601.650,0.816718,N/A
8,56,BAC,46.270,0.776004,N/A
9,90,CFG,49.460,0.770354,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [58]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \n Please try again.')
        portfolio_size = input('Enter the size of your portfolio:')
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:10000000
10000000


In [67]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
    
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,23,ALB,236.050,1.409502,847
1,98,CMA,85.550,1.142082,2337
2,66,BKR,27.400,1.139239,7299
3,89,CF,63.570,1.109087,3146
4,29,AMAT,131.020,1.048395,1526
5,82,CBRE,104.190,1.045079,1919
6,18,AIG,60.010,0.937281,3332
7,24,ALGN,601.650,0.816718,332
8,56,BAC,46.270,0.776004,4322
9,90,CFG,49.460,0.770354,4043


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: